In [1]:
import os
import json
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.layers import InputLayer

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

keras의 backend에서 session을 하나 열어서 만드는 모델의 layers마다 numbering을 한다.<br>
초기화를 하고 싶을 때는, `tf.keras.backend.clear_session()`을 입력하면 된다.

In [2]:
model = Sequential()
# model.add(InputLayer(input_shape=(28, 28, 1))) # InputLayer를 사용할 때는 batch_size는 입력안함.
model.add(Flatten())
model.add(Dense(units=10))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

# model을 build할 때 입력되는 input_shape=(batch_size, width, height, channel)
model.build(input_shape=(None, 28, 28, 1))
# model.build()

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
activation (Activation)      (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
_________________________________________________________________
activation_1 (Activation)    (None, 2)                 0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


In [3]:
class TestModel(Model):
  def __init__(self):
    super(TestModel, self).__init__()
    
    self.flatten = Flatten()
    self.d1 = Dense(units=10)
    self.d1_act = Activation('relu')
    self.d2 = Dense(units=2)
    self.d2_act = Activation('softmax')
    
  def call(self, x):
    x = self.flatten(x)
    x = self.d1(x)
    x = self.d1_act(x)
    x = self.d2(x)
    x = self.d2_act(x)
    return x

model = TestModel()
if not model.built: # model이 build 되었는지 되지 않았는지 확인, 되지 않았다면 input_shape를 입력해서 build.
  model.build(input_shape=(None, 28, 28, 1))

model.summary()

Model: "test_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  7850      
_________________________________________________________________
activation_2 (Activation)    multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  22        
_________________________________________________________________
activation_3 (Activation)    multiple                  0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


model을 build할 때, 데이터 세트를 입력하게 되면 자동적으로 데이터의 shape에 맞게 model이 build되기 때문에 굳이 build()를 쓸 필요는 없지만 learning 전에 분석을 하기 위해 사용할 수 있다. <br>그리고, model이 build가 된 후에는 각 layer에 runnable parameter가 세팅이 되었다는 것이기 때문에 layer의 정보나 처음으로 초기화가 되어 있는 weight, bias의 정보를 확인 할 수 있다.

In [5]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense, Activation

model = Sequential()
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_1_maxpool'))
model.add(Activation('relu', name='conv_1_act'))
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_2_maxpool'))
model.add(Activation('relu', name='conv_2_act'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu', name='dense_1'))
model.add(Dense(units=10, activation='softmax', name='dense_2'))

model.build(input_shape=(None, 28, 28, 1))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
conv_1_maxpool (MaxPooling2D (None, 13, 13, 10)        0         
_________________________________________________________________
conv_1_act (Activation)      (None, 13, 13, 10)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 10)        910       
_________________________________________________________________
conv_2_maxpool (MaxPooling2D (None, 5, 5, 10)          0         
_________________________________________________________________
conv_2_act (Activation)      (None, 5, 5, 10)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 250)              

In [6]:
print(model.layers)

[<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002988ABF3790>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002988AC29970>, <tensorflow.python.keras.layers.core.Activation object at 0x000002988ABF3A90>, <tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000002988ACB2940>, <tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000002988ACB2FD0>, <tensorflow.python.keras.layers.core.Activation object at 0x000002988ACB2B50>, <tensorflow.python.keras.layers.core.Flatten object at 0x000002988ACB2880>, <tensorflow.python.keras.layers.core.Dense object at 0x000002988ACB9520>, <tensorflow.python.keras.layers.core.Dense object at 0x000002988ACB9850>]


In [7]:
print(model.layers[0].get_weights())

[array([[[[-0.15432176,  0.04374015, -0.02467984,  0.11441824,
           0.1295855 ,  0.11015084, -0.10260667, -0.15362588,
          -0.07433225,  0.1484442 ]],

        [[ 0.13954389, -0.13024637,  0.20637068, -0.19673935,
           0.20013154,  0.18219385, -0.12933284,  0.18781403,
           0.02317938,  0.03643373]],

        [[-0.20136449,  0.11340886,  0.09104347, -0.0683336 ,
          -0.10262023, -0.10465235,  0.045892  , -0.12024105,
           0.04060239,  0.22915187]]],


       [[[-0.09256655,  0.08725044, -0.06048971,  0.17439532,
           0.14073181, -0.01200633,  0.12166321,  0.03825903,
          -0.05823302,  0.09250948]],

        [[ 0.06317839,  0.1308744 ,  0.1735183 , -0.13638476,
          -0.04816984,  0.13938025, -0.14548889, -0.14099646,
          -0.06868877, -0.03981078]],

        [[-0.04366478,  0.15068036,  0.09406573,  0.04799721,
          -0.09509957,  0.11919025, -0.09566849,  0.09685758,
          -0.0295471 , -0.0344748 ]]],


       [[[-0.1930